In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt

from pytensor.graph import Apply, Op
from scipy.optimize import approx_fprime

In [ ]:
RANDOM_SEED = 42

def stick_breaking(beta):
    portion_remaining = pt.concatenate([[1], pt.extra_ops.cumprod(1 - beta)[:-1]])
    return beta * portion_remaining

def reparameterize(pi):
    return pi / pi.sum()

def hierarchical_beta(alpha0, beta, nsources, k):
    """Hierarchical Beta distribution for multiple sources."""
    pi_tilt_sources = []
    
    for s in range(nsources):
        beta_params = [(alpha0 * beta[k], alpha0 * (1 - pm.math.sum(beta[:k + 1]))) for k in range(k)]
        pi_tilt = pm.Beta(f'pi_tilt_{s}', 
                          alpha=[b[0] for b in beta_params], 
                          beta=[b[1] for b in beta_params], 
                          dims="component")
        pi_tilt_sources.append(pi_tilt)
    
    #return pm.math.stack(pi_tilt_sources, axis=0)
    return pi_tilt_sources

In [ ]:
import numpy as np
import pytensor.tensor as pt
from pytensor.graph.op import Op, Apply
import pymc as pm
import pandas as pd

# Define interpolate function
def interpolate(x0, y0, x):
    x = np.array(x)
    idx = np.searchsorted(x0, x)
    dl = np.array(x - x0[idx - 1])
    dr = np.array(x0[idx] - x)
    d = dl + dr
    wl = dr / d
    return wl * y0[idx - 1] + (1 - wl) * y0[idx]

# Load data and ensure it's a NumPy array
data = pd.read_csv('grids_example_1.csv', header=None).values
param_min = data[0, :].min()
param_max = data[0, :].max()

# Function to compute log-likelihood    
def my_loglike(x, data):
    x_vals = data[0, :]
    loglike_vals = data[1, :]
    return interpolate(x_vals, loglike_vals, x)

# Define custom PyTensor operation
class LogLike(Op):
    def make_node(self, x, data):
        x = pt.as_tensor_variable(x)  # Ensure x is a tensor
        data = pt.as_tensor_variable(data)  # Ensure data is a tensor
        inputs = [x, data]
        outputs = [pt.TensorType(dtype="float64", broadcastable=(False,))()]
        return Apply(self, inputs, outputs)

    def perform(self, node, inputs, outputs):
        x, data = inputs
        x_vals = data[0, :]
        loglike_vals = data[1, :]
        loglike_eval = interpolate(x_vals, loglike_vals, x)
        outputs[0][0] = np.array(loglike_eval)

# Initialize operation
loglike_op = LogLike()

# PyMC model without observed data
with pm.Model() as no_grad_model:
    # Define prior for x
    x = pm.Uniform("x", lower=param_min, upper=param_max, shape=1)

    # Add a custom potential for the likelihood
    pm.Potential("likelihood", loglike_op(x, data))

    # Sample posterior
    idata = pm.sample(
        200000, 
        tune=5000,
        chains=8,
    )

In [ ]:
import numpy as np
import pytensor.tensor as pt
from pytensor.graph.op import Op, Apply
import pymc as pm
import pandas as pd
import matplotlib.pyplot as plt

# Define interpolate function
def interpolate(x0, y0, x):
    x = np.array(x)
    idx = np.searchsorted(x0, x)
    dl = np.array(x - x0[idx - 1])
    dr = np.array(x0[idx] - x)
    d = dl + dr
    wl = dr / d
    return wl * y0[idx - 1] + (1 - wl) * y0[idx]

# Load data and ensure it's a NumPy array
data = pd.read_csv('grids_example_1.csv', header=None).values
param_min = data[0, :].min()
param_max = data[0, :].max()

In [ ]:
# plot the data
plt.plot(data[0, :], data[1, :], 'o')

plt.show()

In [144]:
import numpy as np
import pandas as pd
import pytensor.tensor as pt
from pytensor.graph.op import Op, Apply
import pymc as pm

# Load profile likelihood data
data = pd.read_csv("profileLikelihoods_NCs_long.csv")

# Define interpolation function
def interpolate(x0, y0, x):
    x = np.array(x)
    
    if x0.size == 0 or y0.size == 0:
        raise ValueError("Empty data passed to interpolation!")

    idx = np.searchsorted(x0, x) - 1  # Ensure index is within bounds
    idx = np.clip(idx, 0, len(x0) - 2)  # Clip to avoid out-of-range errors
    
    dl = x - x0[idx]
    dr = x0[idx + 1] - x
    d = dl + dr

    # Prevent division by zero
    wl = np.where(d != 0, dr / d, 0.5)  
    return wl * y0[idx] + (1 - wl) * y0[idx + 1]

# Define custom likelihood function
class LogLike(Op):
    def make_node(self, βs, num_outcomes, source_data, weights):
        βs = pt.as_tensor_variable(np.asarray(βs))  # Ensure array
        num_outcomes = pt.as_tensor_variable(int(num_outcomes))  # Ensure scalar integer
        source_data = pt.as_tensor_variable(np.asarray(source_data))  # Ensure array
        weights = pt.as_tensor_variable(np.asarray(weights))  # Ensure array

        # The output must be a single scalar
        outputs = [pt.dscalar()]
        return Apply(self, [βs, num_outcomes, source_data, weights], outputs)

    def perform(self, node, inputs, outputs):
        βs, num_outcomes, source_data, weights = inputs
        num_outcomes = int(num_outcomes)  # Ensure integer

        # Compute likelihoods for each outcome
        total_likelihood = np.zeros(num_outcomes)

        for j in range(num_outcomes):
            β_j = βs[j]  # Get β for outcome j

            # Extract outcome-specific x_vals and loglike_vals
            outcome_data = source_data[source_data[:, 2] == (j+1)]  # Filter by outcome index
            
            if outcome_data.shape[0] == 0:
                raise ValueError(f"No data found for outcome {j+1} in source_data!")

            x_vals = outcome_data[:, 0]  # Parameter grid points
            loglike_vals = outcome_data[:, 1]  # Log-likelihood values

            # Interpolate for each component (β_j has n_components)
            likelihoods_j = np.array([interpolate(x_vals, loglike_vals, β) for β in β_j])

            # Weighted sum over components
            total_likelihood[j] = np.dot(weights, likelihoods_j)

        # Sum over all outcomes to get the final likelihood
        final_likelihood = np.sum(total_likelihood)

        # Ensure the output is a scalar
        outputs[0][0] = np.array(final_likelihood)

# Initialize the custom operation
loglike_op = LogLike()

# Simulated test case for source 1
n_outcomes_test = int(data["outcome"].nunique())  # Ensure integer
n_components = 3  # Assume 3 components for illustration

βs_test = np.random.uniform(low=-1, high=-0.5, size=(n_outcomes_test, n_components))  # Simulated β values
num_outcomes_test = int(data[data["source"] == 1]["outcome"].nunique())  # Ensure integer
source_1_data = data[data["source"] == 1][["point", "value", "outcome"]].values  # Include outcome column
weights_test = np.full(n_components, 1 / n_components)  # Equal mixture weights


test_out = loglike_op(βs_test, num_outcomes_test, source_1_data, weights_test)
print("Evaluated Test Likelihood Output:", test_out.eval())





Evaluated Test Likelihood Output: -29.8796475201578
